In [84]:
# 1.a)

from pgmpy.models import DiscreteBayesianNetwork # ImportError: BayesianNetwork has been deprecated. Please use DiscreteBayesianNetwork instead.
from pgmpy.factors.discrete import TabularCPD

model=DiscreteBayesianNetwork([('S', 'O'), ('S', 'L'), ('S', 'M'), ('L', 'M')])

# P(S = 1) = 0.4, P(S = 0) = 0.6
CPD_S=TabularCPD(variable='S',variable_card= 2,values= [[0.6], [0.4]])

# P(O = 1∣S = 1) = 0.7, P(O = 1∣S = 0) = 0.1
CPD_O=TabularCPD(variable='O', variable_card=2,values=[[0.9, 0.3], [0.1, 0.7]],evidence=['S'], evidence_card=[2])

#  P(L = 1∣S = 1) = 0.8, P(L = 1∣S = 0) = 0.3
CPD_L=TabularCPD(variable='L',variable_card=2,values=[[0.7, 0.2],[0.3, 0.8]], evidence=['S'], evidence_card=[2])

# P(M = 1∣S = 1, L = 1) = 0.9, P(M = 1∣S = 1, L = 0) = 0.5
# P(M = 1∣S = 0, L = 1) = 0.6, P(M = 1∣S = 0, L = 0) = 0.2
CPD_M=TabularCPD(variable='M',variable_card=2,values=[[0.8, 0.4, 0.9, 0.5], [0.2, 0.6, 0.1, 0.5]], evidence=['S', 'L'], evidence_card=[2, 2])

model.add_cpds(CPD_S,CPD_O,CPD_L,CPD_M)

model.check_model()

print(model.local_independencies(['O','L','M']))

(O ⟂ M, L | S)
(L ⟂ O | S)
(M ⟂ O | S, L)


In [85]:
#1.b)

# probabilitati
P_S = {0:0.6, 1:0.4}
P_O_stiind_S = {0:{0:0.9,1:0.1}, 1:{0:0.3,1:0.7}}
P_L_stiind_S = {0:{0:0.7,1:0.3}, 1:{0:0.2,1:0.8}}
P_M_stiind_SL = {(0,0):{0:0.8,1:0.2}, (0,1):{0:0.4,1:0.6}, (1,0):{0:0.5,1:0.5}, (1,1):{0:0.1,1:0.9}}

o = 1  # contine "offer"
l = 1  # are link
m = 1  # email lung

# calculam scorul pentru spam si non-spam
scor0 = P_S[0]*P_O_stiind_S[0][o]*P_L_stiind_S[0][l]*P_M_stiind_SL[(0,l)][m]
scor1 = P_S[1]*P_O_stiind_S[1][o]*P_L_stiind_S[1][l]*P_M_stiind_SL[(1,l)][m]

total = scor0 + scor1
p0 = scor0/total
p1 = scor1/total

if p1>p0:
    pred = 1
else:
    pred = 0

print("P(Non-Spam) =", p0)
print("P(Spam) =", p1)
if pred==1:
    print("SPAM")
else:
    print("NON-SPAM")


P(Non-Spam) = 0.05084745762711865
P(Spam) = 0.9491525423728814
SPAM


In [86]:
# 2


from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([('Dice', 'Ball')])

cpd_dice = TabularCPD('Dice', 6, [[1/6],[1/6],[1/6],[1/6],[1/6],[1/6]])

cpd_ball = TabularCPD('Ball', 3,
    [
        [3/10,3/10,3/10,3/10,3/10,4/10],  # rous
        [4/10,4/10,4/10,5/10,4/10,4/10],  # albastru
        [3/10,3/10,3/10,2/10,3/10,2/10]   # negru
    ],
    evidence=['Dice'], evidence_card=[6]
)

model.add_cpds(cpd_dice, cpd_ball)

model.check_model()

tab=VariableElimination(model)
res=tab.query(['Ball'])
print(res)


+---------+-------------+
| Ball    |   phi(Ball) |
+=========+=============+
| Ball(0) |      0.3167 |
+---------+-------------+
| Ball(1) |      0.4167 |
+---------+-------------+
| Ball(2) |      0.2667 |
+---------+-------------+
